In [6]:
# %pip install transformers
# %pip install sentence_transformers
# %pip install sentencepiece
# %pip install soundfile pyctcdecode==v0.1.0
# %pip install https://github.com/kpu/kenlm/archive/master.zip
# %pip install torch torchvision torchaudio

# 2. Speech to text

In [2]:
import os, zipfile
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import soundfile as sf
import torch
import kenlm
from pyctcdecode import Alphabet, BeamSearchDecoderCTC, LanguageModel
import IPython
import numpy as np
import matplotlib.pyplot as plt
import pyaudio
import wave
from gtts import gTTS
import requests
from playsound import playsound

playsound is relying on another python subprocess. Please use `pip install pygobject` if you want playsound to run more efficiently.


In [3]:
# Load model directly
model_dir = "./models/nguyenvulebinh--wav2vec2-base-vietnamese-250h/snapshots/69e9000591623e5a4fc2f502407860bcdc0de0b2"
# load model and tokenizer
# processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
# model = Wav2Vec2ForCTC.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h")
processor = Wav2Vec2Processor.from_pretrained(model_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_dir)


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


/home/xuanai/Python-env/library_robot/lib/python3.8/site-packages/transformers/configuration_utils.py:381: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at ./models/nguyenvulebinh--wav2vec2-base-vietnamese-250h/snapshots/69e9000591623e5a4fc2f502407860bcdc0de0b2 were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC fro

In [4]:
def get_decoder_ngram_model(tokenizer, ngram_lm_path):
    vocab_dict = tokenizer.get_vocab()
    sort_vocab = sorted((value, key) for (key, value) in vocab_dict.items())
    vocab = [x[1] for x in sort_vocab][:-2]
    vocab_list = vocab
    # convert ctc blank character representation
    vocab_list[tokenizer.pad_token_id] = ""
    # replace special characters
    vocab_list[tokenizer.unk_token_id] = ""
    # vocab_list[tokenizer.bos_token_id] = ""
    # vocab_list[tokenizer.eos_token_id] = ""
    # convert space character representation
    vocab_list[tokenizer.word_delimiter_token_id] = " "
    # specify ctc blank char index, since conventially it is the last entry of the logit matrix
    alphabet = Alphabet.build_alphabet(vocab_list, ctc_token_idx=tokenizer.pad_token_id)
    lm_model = kenlm.Model(ngram_lm_path)
    decoder = BeamSearchDecoderCTC(alphabet,
                                   language_model=LanguageModel(lm_model))
    return decoder

lm_file = './models/vi_lm_4grams.bin'
ngram_lm_model = get_decoder_ngram_model(processor.tokenizer, lm_file)

In [5]:
# define function to read in sound file
def map_to_array(batch):
    speech, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech
    batch["sampling_rate"] = sampling_rate
    return batch
# speech to text function
def speech2text(audio_file):
  # load dummy dataset and read soundfiles
  ds = map_to_array({"file": audio_file})
  # infer model
  input_values = processor(
        ds["speech"],
        sampling_rate=ds["sampling_rate"],
        return_tensors="pt"
  ).input_values
  # ).input_values.to("cuda")
  # model.to("cuda")
  logits = model(input_values).logits[0]
  # print(logits.shape)
  # decode ctc output
  pred_ids = torch.argmax(logits, dim=-1)
  greedy_search_output = processor.decode(pred_ids)
  beam_search_output = ngram_lm_model.decode(logits.cpu().detach().numpy(), beam_width=500)
  print("Greedy search output: {}".format(greedy_search_output))
  print("Beam search output: {}".format(beam_search_output))
  return greedy_search_output

In [ ]:
# audio_file = '/content/gdrive/MyDrive/Intern/audio/ban_co_the_16kHz.wav'
audio_file = 'audio-test/t1_0001-00010.wav'
text = speech2text(audio_file)
print(text)
IPython.display.Audio(filename=audio_file, autoplay=True)

In [12]:
def record_audio(file_path, duration=5, sample_rate=16000, chunk=1024, format=pyaudio.paInt16, channels=1):
    audio = pyaudio.PyAudio()
    stream = audio.open(format=format, channels=channels,
                        rate=sample_rate, input=True,
                        frames_per_buffer=chunk)

    print("..... Recording .....", end='\r')
    frames = []

    for i in range(0, int(sample_rate / chunk * duration)):
        data = stream.read(chunk)
        frames.append(data)

    print(" " * 50, end="\r", flush=True)

    stream.stop_stream()
    stream.close()
    audio.terminate()

    with wave.open(file_path, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(audio.get_sample_size(format))
        wf.setframerate(sample_rate)
        wf.writeframes(b''.join(frames))

# Usage example:
# Specify the file path where you want to save the recorded audio file
file_path = 'recorded_audio.wav'
# Record audio for 5 seconds (you can adjust the duration if needed)
record_audio(file_path, duration=1, sample_rate=16000)



ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card


In [13]:
audio_file = 'recorded_audio.wav'
# Record audio for 5 seconds (you can adjust the duration if needed)
record_audio(audio_file, duration=5, sample_rate=16000)
speech2text(audio_file)
IPython.display.Audio(filename=audio_file, autoplay=True)

ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card


Greedy search output: a lô                        
Beam search output: a lô


# 3. Question Answering

In [ ]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
model = SentenceTransformer("vinai/phobert-base")
from tqdm import tqdm

In [76]:
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
model = SentenceTransformer("vinai/phobert-base")
from tqdm import tqdm

No sentence-transformers model found with name /home/xuanai/.cache/torch/sentence_transformers/vinai_phobert-base. Creating a new one with MEAN pooling.


In [77]:
Ques_Ans= {"Trí tuệ nhân tạo là gì?" : "Trong khoa học máy tính, trí tuệ nhân tạo hay AI (tiếng Anh: artificial intelligence), đôi khi được gọi là trí thông minh nhân tạo, là trí thông minh được thể hiện bằng máy móc, trái ngược với trí thông minh tự nhiên của con người. Thông thường, thuật ngữ trí tuệ nhân tạo thường được sử dụng để mô tả các máy chủ móc (hoặc máy tính) có khả năng bắt chước các chức năng nhận thức mà con người thường phải liên kết với tâm trí, như học tập và giải quyết vấn đề.",
           "Có bao nhiêu châu lục trên thế giới?" : "Theo quy ước, Trái Đất có 7 lục địa bao gồm Á, Phi, Nam Mỹ, Bắc Mỹ, Âu, Úc và Nam Cực",
           "Có bao nhiêu cung hoàng đạo?" : "Thứ tự của 12 cung Hoàng Đạo là Bạch Dương, Kim Ngưu, Song Tử, Cự Giải, Sư Tử, Xử Nữ, Thiên Bình, Thiên Yết, Nhân Mã, Ma Kết, Bảo Bình và Song Ngư. Mỗi khu vực được đặt tên theo chòm sao mà nó đi qua trong lúc được đặt tên."}

In [78]:
question_list= list(Ques_Ans.keys())

In [79]:
def cosine_similarity(x1, x2):
  numerator = np.dot(x1,x2.T)
  denominator = np.linalg.norm(x1)*np.linalg.norm(x2)
  return numerator/denominator

In [80]:
corpus_embeddings = model.encode(list(Ques_Ans.keys()))

In [81]:
term_matrix= {}
for ques, ans in tqdm(Ques_Ans.items()):
  embedding = model.encode(ques)
  term_matrix[(ques, ans)] = embedding

100%|██████████| 3/3 [00:00<00:00, 22.83it/s]


In [82]:
def get_ans(query, term_matrix):
  embedding = model.encode(query)
  ranking = []
  for doc_info, embed in tqdm(term_matrix.items()):
    score = cosine_similarity(embedding, embed)
    ranking.append((score, doc_info[1]))
    ans= max(ranking)
  if ans[0] > 0.5:
     print(ans)
  else:
     print("Không tìm thấy câu trả lời")

In [83]:
query= 'bao nhiêu cung hoàng đạo'
get_ans(query, term_matrix)

100%|██████████| 3/3 [00:00<00:00, 4003.47it/s]

(0.6439748, 'Thứ tự của 12 cung Hoàng Đạo là Bạch Dương, Kim Ngưu, Song Tử, Cự Giải, Sư Tử, Xử Nữ, Thiên Bình, Thiên Yết, Nhân Mã, Ma Kết, Bảo Bình và Song Ngư. Mỗi khu vực được đặt tên theo chòm sao mà nó đi qua trong lúc được đặt tên.')


### Sumarization

In [1]:
from transformers import pipeline

summarizer = pipeline("summarization", model="vinai/bartpho-syllable")

In [2]:
summarizer('The tower is 324 metres (1,063 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest man-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure to reach a height of 300 metres. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct.')

/home/xuanai/Desktop/Intern/Voice_assitant/.venv/lib/python3.8/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'summary_text': 'The tower is 324 metres (1,063 ft) tall, about the'}]

In [3]:
src = "Theo BHXH Việt Nam, nhiều doanh nghiệp vẫn chỉ đóng BHXH cho người lao động theo mức lương.\
Dù quy định từ 1/1/2018, tiền lương tháng đóng BHXH gồm mức lương và thêm khoản bổ sung khác. \
BHXH Việt Nam vừa có báo cáo về tình hình thực hiện chính sách BHXH thời gian qua. \
Theo đó, tình trạng nợ, trốn đóng BHXH, BHTN vẫn xảy ra ở hầu hết các tỉnh, thành. \
Thống kê tới ngày 31/12/2020, tổng số nợ BHXH, BHYT, BHTN là hơn 13.500 tỷ đồng, \
chiếm 3,35 % số phải thu, trong đó: Số nợ BHXH bắt buộc là hơn 8.600 tỷ đồng, \
nợ BHTN là 335 tỷ đồng. Liên quan tới tiền lương đóng BHXH, báo cáo của \
BHXH Việt Nam cho thấy: Nhiều doanh nghiệp vẫn chủ yếu xây dựng thang, \
bảng lương để đóng BHXH bằng mức thấp nhất. Tức là bằng mức lương tối \
thiểu vùng, cộng thêm 7 % đối với lao động đã qua đào tạo nghề và cộng \
thêm 5 % hoặc 7 % đối với lao động làm nghề hoặc công việc nặng nhọc, \
độc hại, nguy hiểm, đặc biệt nặng nhọc độc hại và nguy hiểm. Đối với \
lao động giữ chức vụ, khoảng 80 % doanh nghiệp đã xây dựng thang, \
bảng lương cụ thể theo chức danh. Đơn cử như với chức vụ giám đốc \
sản xuất, giám đốc điều hành, trưởng phòng. Còn lại các doanh nghiệp \
xây dựng đối với lao động giữ chức vụ theo thang lương, bảng lương \
chuyên môn nghiệp vụ và bảng phụ cấp chức vụ, phụ cấp trách nhiệm. \
Thống kê của BHXH Việt Nam cũng cho thấy, đa số doanh nghiệp đã đăng \
ký đóng BHXH cho người lao động theo mức lương mà không có khoản bổ \
sung khác. Mặc dù quy định từ ngày 1/1/2018, tiền lương tháng đóng BHXH \
gồm mức lương và thêm khoản bổ sung khác."

In [4]:
vni= summarizer(src, max_length= 100)
vni

[{'summary_text': '(Ảnh minh hoạ). (Ảnh: PV / Vietnam +). Theo BHXH Việt Nam, nhiều doanh nghiệp vẫn chỉ đóng BHXH cho người lao động theo mức lương.Dù quy định từ 1/1/2018, tiền lương tháng đóng BHXH gồm mức lương và thêm khoản bổ sung khác. BHXH Việt Nam vừa có báo cáo về tình hình thực hiện chính sách BHXH thời gian qua. Theo đó, tình trạng nợ, trốn đóng BHXH, BH'}]

# 4. Text to speech

## 4.1 Python library

In [14]:
def text2speech(text):
  # Language in which you want to convert
  language = 'vi'
  # Passing the text and language to the engine, here we have specified slow=False
  tts = gTTS(text=text, lang=language, slow=False)
  # Save the converted audio to a file
  tts.save("output.mp3")
  playsound('output.mp3')

In [15]:
# Vietnamese text to be converted to speech
vietnamese_text = "Xin chào, đây là ví dụ về chuyển văn bản thành tiếng nói bằng Python."
text2speech(vietnamese_text)
# IPython.display.Audio(filename="output.mp3", autoplay=True)
playsound('output.mp3')

## 4.2 Zalo API

In [16]:
# Set your API key and input text
api_key = "W3bgV7wf6lKxv9SNtZspLKE8s51MNmvk"
input_text = "Xin chào, đây là ví dụ về chuyển văn bản thành tiếng nói ."

# API endpoint URL
url = "https://api.zalo.ai/v1/tts/synthesize"

# Headers containing the API key
headers = {
    "apikey": api_key
}
    
# Data to be sent in the POST request
data = {
    "input": input_text
}

# Send POST request to the API endpoint
response = requests.post(url, headers=headers, data=data)
print(response.text)

# Parse the JSON response
json_response = response.json()

output_file = 'outpu.wav'
# Check if the response contains the URL
if "data" in json_response and "url" in json_response["data"]:
    audio_url = json_response["data"]["url"]

    # Download the audio file
    audio_response = requests.get(audio_url)

    # Save the audio content to a WAV file
    with open(output_file, "wb") as wav_file:
        wav_file.write(audio_response.content)
        print(f"Audio file saved as {output_file}")
else:
    print("Error: Unable to fetch audio URL from the API response.")

playsound(output_file)

{"error_code":0,"error_message":"Successful.","data":{"url":"https://chunk.lab.zalo.ai/53c84d34005ae904b04b/53c84d34005ae904b04b"}}
Audio file saved as outpu.wav


KeyboardInterrupt: 

### Scale audio speed

In [32]:
import wave
import numpy as np
from scipy.io.wavfile import write


def scale_audio_speed(audio_file, scale_speed):
    audio = wave.open(audio_file, 'rb')
    sample_freq = audio.getframerate()
    n_samples = audio.getnframes()
    signal_wave = audio.readframes(n_samples)
    signal_array = np.frombuffer(signal_wave, dtype=np.int16)
    write('test.wav', int(sample_freq * scale_speed), signal_array)


In [70]:
scale_audio_speed('recorded_audio.wav', 1.5)

# Rasa voice assitant

In [17]:
import requests
def get_rasa_response(user_query):
    rasa_server_url = "http://192.168.0.36:5005/webhooks/rest/webhook"
    response = requests.post(rasa_server_url, json={"message": user_query})
    rasa_response = response.json()
    return rasa_response

# response = get_rasa_response('hello mình tên là ái')
# print(response[0]['text'])

In [3]:
while(1):
    mess = input("User: ")
    print(mess)
    response = get_rasa_response(mess)
    print(response)


hello
[{'recipient_id': 'default', 'text': 'Hello None!'}]
how are you
[{'recipient_id': 'default', 'text': 'Hey None, bạn có khỏe không?', 'buttons': [{'title': 'great', 'payload': '/mood_great'}, {'title': 'sad', 'payload': '/mood_unhappy'}]}]
bạn có thể làm gì
[{'recipient_id': 'default', 'text': 'Tuyệt, vậy thì tốt quá.'}]


In [18]:
audio_file = 'recorded_audio.wav'

while(1):
    record_audio(audio_file, duration=5, sample_rate=16000)
    mess = speech2text(audio_file)
    print('User: ', mess)
    
    response = get_rasa_response(mess)
    for i in range(len(response)):
        txt = list(response[i].values())[1]
        print(txt)
        text2speech(txt)

ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_route.c:869:(find_matching_chmap) Found no matching channel map
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card


KeyboardInterrupt: 

In [5]:
mess = [{'recipient_id': 'default', 'text': 'OK! Bạn muốn mượn sách trong một ngày.'}, {'recipient_id': 'default', 'text': 'Tôi sẽ cho bạn mượn cuốn sách vật lý trong một ngày.'}]
for i in range(len(mess)):
    print(list(mess[i].values())[1])

OK! Bạn muốn mượn sách trong một ngày.
Tôi sẽ cho bạn mượn cuốn sách vật lý trong một ngày.
